In [1]:
import ccxt
import datetime
import time
import pandas as pd
import sqlite3

exchange = ccxt.binance()


def calculate_average_price_for_volume(orders, target_volume=1):
    accumulated_volume = 0
    total_value = 0
    
    for price, volume in orders:
        
        if accumulated_volume + volume >= target_volume:
            remaining_volume = target_volume - accumulated_volume
            total_value += price * remaining_volume
            accumulated_volume += remaining_volume
            break
        
        else:
            total_value += price * volume
            accumulated_volume += volume
    
    # If we haven't reached the target volume, return None
    if accumulated_volume < target_volume:
        return None

    return total_value / target_volume


In [ ]:
# Establish a connection to SQLite database
conn = sqlite3.connect('orderbook_data.db')
cur = conn.cursor()

# Create a table if it doesn't exist
cur.execute('''CREATE TABLE IF NOT EXISTS btc_orderbook_data (
                datetime TEXT,
                avg_highest_bid_price REAL,
                avg_lowest_ask_price REAL,
                bid_ask_spread REAL
            )''')

cur.execute('''CREATE TABLE IF NOT EXISTS eth_orderbook_data (
                datetime TEXT,
                avg_highest_bid_price REAL,
                avg_lowest_ask_price REAL,
                bid_ask_spread REAL
            )''')



In [ ]:
def fetch_and_store_orderbook_data(symbol):
    
    data = exchange.fetchOrderBook(symbol)
    bids = data['bids']
    asks = data['asks']

    average_highest_bid_price = calculate_average_price_for_volume(bids, 1)
    average_lowest_ask_price = calculate_average_price_for_volume(asks, 1)

    bid_ask_spread = average_lowest_ask_price - average_highest_bid_price

    row = {'datetime': pd.to_datetime(data['datetime']).strftime('%Y-%m-%d %H:%M:%S'),
            'avg_highest_bid_price': average_highest_bid_price,
            'avg_lowest_ask_price': average_lowest_ask_price,
            'bid_ask_spread': bid_ask_spread}

    if symbol == 'BTC/USDT:USDT':
        table_name = 'btc_orderbook_data'
    elif symbol == 'ETH/USDT:USDT':
        table_name = 'eth_orderbook_data'

    # Save data to SQLite database
    cur.execute(f'''INSERT INTO {table_name} (datetime, avg_highest_bid_price, avg_lowest_ask_price, bid_ask_spread)
                    VALUES (?, ?, ?, ?)''', (row['datetime'], row['avg_highest_bid_price'], row['avg_lowest_ask_price'], row['bid_ask_spread']))
    
    conn.commit()




symbols = ['BTC/USDT:USDT', 'ETH/USDT:USDT']

while True:
    
    for symbol in symbols:
        fetch_and_store_orderbook_data(symbol)
    
    time.sleep(60)
    

# Close the database connection
conn.close()